# TODO

- Note that second `generate_figure()` function is the one used in the paper
- [ ] Refactor plotting functions so that each subfigure has its own function call
    - This makes composite generation cleaner
- [ ] Separate computation from plotting
    - [ ] Rebuild composite script for generating figure for each data file
- [ ] Improve comments
    - [ ] Note that `fig.patch.set_alpha(0)` is necessary to overlay composite figure in Inkscape

# High-resolution line scan analysis

A notebook for comparing the 'secondary electron concurrent' data with the CL intensity for each pixel in a line scan .h5 file

__Assumptions__:
- An Odemis .h5 file that was obtained as a `nx1` CL map corresponding to a line scan across a given feature.

In [3]:
import numpy as np
import pandas as pd
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec
from mpl_toolkits import axes_grid1
from matplotlib import transforms
from scipy.signal import find_peaks, savgol_filter, butter, filtfilt
from scipy.ndimage import median_filter
from scipy.ndimage.filters import gaussian_filter1d
from scipy.optimize import curve_fit
from skimage.restoration import denoise_wavelet
from sklearn.decomposition import NMF, PCA
from glob import glob
import os

blues = mpl.colors.LinearSegmentedColormap.from_list('', ['black','tab:blue','white'])
reds = mpl.colors.LinearSegmentedColormap.from_list('', ['black','tab:red', 'white'])
greens = mpl.colors.LinearSegmentedColormap.from_list('', ['black', 'tab:green','white'])
oranges = mpl.colors.LinearSegmentedColormap.from_list('', ['black','tab:orange','white'])
purples = mpl.colors.LinearSegmentedColormap.from_list('', ['black','purple','white'])
golds = mpl.colors.LinearSegmentedColormap.from_list('', ['black','gold','white'])
grays = mpl.colors.LinearSegmentedColormap.from_list('', ['black','gray','white'])
RGB = mpl.colors.LinearSegmentedColormap.from_list('', ['gray', 'blue',  'green', 'red', 'gray'])

mpl.rcParams.update(mpl.rcParamsDefault) # Reset user themes
plt.style.use(['science']) # Figures sized for publication
plt.style.use(['notebook']) # Figures adjusted for distant viewing

cs = ['tab:red', 'tab:green', 'tab:blue', 'tab:orange', 'purple', 'gold', 'gray']

# Quick functions for picking out relevant parameters from Odemis files
get_img_arr = lambda x : x['ImageData']['Image'][()].squeeze()
get_scaleX = lambda x : x['ImageData']['DimensionScaleX'][()].squeeze()
get_scaleY = lambda x : x['ImageData']['DimensionScaleY'][()].squeeze()
get_scaleC = lambda x : x['ImageData']['DimensionScaleC'][()].squeeze() # for wavelengths

# Quick functions for cleaning data
normalize = lambda x : (x-np.amin(x))/(np.amax(x)-np.amin(x))
rescale = lambda x : x/np.amax(x)

def remove_spikes(x, threshold=50):
    x = x.copy()
    peaks, props = find_peaks(x, threshold=threshold)
    for i in peaks:
        x[i-2:i+3] = np.mean(np.concatenate([x[i-3:i-5],x[i+4:i+6]]))
    return x

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


# Functions for displaying SEM image data
def get_ranges(acq, mult_scale=1e6):
    """
    Get points along x and y dimensions for given acquisition.
    Parameters
    ----------
    acq : hdf5 group
        
    mult_scale : float
         (Default value = 1e6)

    Returns
    -------
    x_rng, y_rng : tuple
    """
    arr = acq['ImageData']['Image'][()].squeeze()
    title = acq['PhysicalData']['Title'][()]
    
    if arr.ndim == 1:
        arr = np.atleast_2d(arr).T  # Extend nx1 datasets if necessary
    
    if b'Spectrum' not in title:
#         try:
        ny, nx = arr.shape[-2:] # need to exlude wl dim for Acq 2
#         except ValueError:  # catch case for nx1 dataset
#             ny = arr.shape[-1]
#             nx = 1
    elif arr.ndim == 2:  # Acq 2 during nx1 dataset
        ny = arr.shape[-1]
        nx = 1
    else:
        ny, nx = arr.shape[-2:]
    
    yscale = acq['ImageData']['DimensionScaleY'][()]*mult_scale 
    xscale = acq['ImageData']['DimensionScaleX'][()]*mult_scale 
    yoff = acq['ImageData']['YOffset'][()]*mult_scale
    xoff = acq['ImageData']['XOffset'][()]*mult_scale
    
    ylen = ny*yscale
    xlen = nx*xscale
    x0 = -xlen/2 + xoff
    x1 = xlen/2 + xoff
    y0 = -ylen/2 + yoff
    y1 = ylen/2 + yoff
    
    x_rng = np.arange(x0+xscale/2, x1, xscale)
    y_rng = np.arange(y0+yscale/2, y1, yscale)[::-1] # for proper xarray coords
    
    return x_rng, y_rng

def add_colorbar(im, aspect=20, pad_fraction=0.5, **kwargs): # See https://nbviewer.jupyter.org/github/mgeier/python-audio/blob/master/plotting/matplotlib-colorbar.ipynb
    divider = axes_grid1.make_axes_locatable(im.axes)
    width = axes_grid1.axes_size.AxesY(im.axes, aspect=1./aspect)
    pad = axes_grid1.axes_size.Fraction(pad_fraction, width)
    current_ax = plt.gca()
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.sca(current_ax)
    return im.axes.figure.colorbar(im, cax=cax, **kwargs)

# From rectangles to list of points.

def enume(htuples):
    point_list = []
    for htuple in htuples:
        x_coords = list(range(min(htuple[0][0], htuple[1][0]), max(htuple[0][0], htuple[1][0]) + 1))
        y_coords = list(range(min(htuple[0][1], htuple[1][1]), max(htuple[0][1], htuple[1][1]) + 1))
        for y in y_coords:
            for x in x_coords:
                point_list.append([x,y])
    return point_list
                
# From coordinates to index in spec_list

def coord_to_index(x,y):
    return y * x_width + x 

### Read in the file

In [2]:
# Glob a list of files matching these keywords
files = sorted(glob('data/*.h5'))
# Set the path to one specific file in the above list
p = files[0]
f = h5py.File(p, 'r')
print(f)

<HDF5 file "hBN_s1_loc2x2-9x1_linescan1_05keV_1000pA_20kx_1B_4G_100ms_LV30_799x1f.h5" (mode r)>


In [ ]:
# Import spectra from linescan
f = h5py.File(p, 'r')
cl = get_img_arr(f['Acquisition2'])  # size (1024,n) array
cl = np.array([remove_spikes(s, threshold=threshold) for s in cl])
cl_full = np.atleast_3d(cl)
sec = get_img_arr(f['Acquisition1'])  # Secondary electrons concurrent
sec = normalize(sec)
# sec = sec/1e3

# Generate x-axis scale and total wavelength sums
wavelengths = get_scaleC(f['Acquisition2'])*1e9
w_sums = [np.sum(i) for i in cl]
w_sums = np.array(w_sums)/1e6

step_x = get_scaleX(f['Acquisition2'])  # distance b/w pixels, in m
distances = np.array([i*step_x for i in range(len(cl[0]))])*1e6  # populate scaled x-axis
    
# Pick out shape parameters from acquisition array, then reshape
w_width = cl_full.shape[0]  # wavelengths, size 1024 for CCD pixels on spectrometer
y_width = cl_full.shape[1]  # Number of acquisitions in horizontal direction (even though says y)
x_width = cl_full.shape[2]
pixels = y_width * x_width
spectra = cl_full.transpose(1,2,0).reshape(pixels, w_width)  # Reshape so that row is one spectrum

In [ ]:
# Inspect spectra before/after subtraction
inspect_x, inspect_y = 600, 0
inspect_idx = coord_to_index(inspect_x,inspect_y)
distance = distances[inspect_idx]

fig, ax = plt.subplots()
ax.plot(wavelengths, spectra[inspect_idx], c='purple', label='Original spectra')
ax.plot(wavelengths, line_bg, c='r', linestyle='--', label='Adjusted background')
ax.set_title(f'Line spectra at {distance} $\mu$m')
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Counts')
ax.set_xlim(350, 900)
ax.set_ylim(0, 800)
ax.legend()
plt.show()

In [ ]:
# Display heatmap of spectra
XX, YY = np.meshgrid(wavelengths, distances)
fig, ax = plt.subplots()
heatmap = ax.pcolormesh(XX, YY, spectra_corrected, shading='auto', cmap='RdBu_r')
ax.set_ylabel('Distance along scan ($\mu$m)')
ax.set_xlabel('Wavelength (nm)')
# ax.set_xlim(wl_min,wl_max)
ax.set_xlim(350,900)
plt.show()

## Orientation: horizontal linescan

In [ ]:
# Generate composite figure

wl_min = 350  # nm, x_min for displayed wavelength axis
wl_max = 950  # nm, x_max for displayed wavelength axis
# cmap1 = mpl.cm.get_cmap('gist_stern')
cmap1 = mpl.cm.get_cmap('RdBu_r')
nmf_focus = 1
start_wl = 750
end_wl = 900
threshold = 40

# Convert 'start_wl' and 'end_wl' to raw index values of wavelengths array
start = find_nearest(wavelengths, start_wl)
end = find_nearest(wavelengths, end_wl)

# Gather secondary electron survey data
ses = get_img_arr(f['Acquisition0'])
(ny, nx) = np.shape(ses)  # number of pixels (y,x) of SEM image
ses_x_step = get_scaleX(f['Acquisition0'])
ses_y_step = get_scaleY(f['Acquisition0'])

# Gather CL acquisition reference parameters
cl_acq = f['Acquisition2']
x_center = ses_x_step*nx/2
y_center = ses_y_step*ny/2
y_rng, x_rng = get_ranges(cl_acq, mult_scale=1)  # This is backwards from what I expect, but plots correctly
cl_x = [x_center + np.amin(x_rng), x_center + np.amax(x_rng)]
cl_y = [y_center + np.amin(y_rng), y_center + np.amax(y_rng)]

x_img = np.linspace(0, nx*ses_x_step, nx)  # Raw distance of image width
y_img = np.linspace(0, ny*ses_y_step, ny)  # Raw distance of image height
xx, yy = np.meshgrid(x_img, y_img)


# Generate x-axis scale and total wavelength sums
# wavelengths = get_scaleC(f['Acquisition2'])*1e9
# w_sums = [np.sum(i) for i in spectra_corrected]
# w_sums = np.array(w_sums)
w_sums = rescale(w_sums)

# Generate cummulative intensity for all wavelengths between start and end
I_tot = np.array([line.sum() for line in spectra_corrected])  # Sum the intensities over total wavelengths
I_sub = np.array([line[start:end].sum() for line in spectra_corrected])  # Sum the intensities in a given index range
I_tot = normalize(I_tot)
I_sub = normalize(I_sub)
# I_tot = I_tot/1e3
# I_sub = I_sub/1e3

# Pick out shape parameters from acquisition array, then reshape
# w_width = cl_full.shape[0]  # wavelengths, size 1024 for CCD pixels on spectrometer
# y_width = cl_full.shape[1]  # Number of acquisitions in horizontal direction (even though says y)
# x_width = cl_full.shape[2]
# pixels = y_width * x_width

# Generate colormesh params from file metadata
# step_x = get_scaleX(f['Acquisition2'])  # distance b/w pixels, in m
# distances = np.array([i*step_x for i in range(len(spectra_corrected[0]))])*1e6  # populate scaled x-axis
XX, YY = np.meshgrid(distances, wavelengths)

# Specify parameters for highlighted subset of NMF components (graph inset)
wl_min_subset = wl_min + (1/3 - 0.02)*(wl_max - wl_min)
wl_max_subset = wl_max
sub_start = find_nearest(wavelengths, wl_min_subset)
sub_end = find_nearest(wavelengths, wl_max_subset)

# START PLOT -------------------------------

fig = plt.figure(figsize=(10,8))
gs = GridSpec(ncols=5, nrows=4, figure=fig, height_ratios=[3,3,9,3], width_ratios=[1,1.25,1.75,2,12])
#     fig.suptitle('CL Spectra')

# Secondary Electron Survey with Line Scan
ax6 = fig.add_subplot(gs[0:2,0:-1])
ax6.pcolormesh(xx, yy, ses, cmap='gray', shading='auto', alpha=1, vmin=np.amin(ses)*1.02, vmax=np.amax(ses)*0.98)  # SEM image
#     ax6.imshow(ses, cmap='gray', alpha=0.4)  # SEM image
ax6.plot(cl_x, cl_y, linewidth=3, c="white")  # Line overlay of acquisition region
ax6.invert_yaxis()
ax6.axis('equal')  # Force aspect ratio for SEM image
ax6.axis('off')

# NMF Spatial Components
ax5 = fig.add_subplot(gs[0,-1])
for i in range(num_NMF_components):
    ax5.plot(distances, W_nmf[:,i].reshape(y_width, x_width), c=cs[i], linewidth=2, alpha=0.4)
ax5.plot(distances, W_nmf[:,nmf_focus].reshape(y_width, x_width), c=cs[nmf_focus], linewidth=2)
ax5.set_xlim(np.amin(distances), np.amax(distances))
ax5.set_xlabel('Distance along scan ($\mu$ m)')
ax5.xaxis.set_visible(False)
ax5.set_ylabel('NMF Comp.\nStrength (a.u.)')
ax5.yaxis.set_label_position("right")
ax5.yaxis.tick_right()
#     ax5.legend()

# Cummulative CL intensity vs position
ax3 = plt.subplot(gs[1,-1])
ax3.fill_between(x=distances, y1=I_sub, y2=I_tot, color="C3", alpha=0.4)  # Int wavelengths
ax3.fill_between(x=distances, y1=I_tot, y2=np.amin(I_tot), alpha=0.4)  # All wavelengths
ax3.set_xlim(np.amin(distances), np.amax(distances))
ax3.xaxis.set_visible(False)
ax3.set_ylim(np.amax(I_sub), np.amin(I_sub))
ax3.set_ylabel('Normalized\nIntensity')
ax3.yaxis.set_label_position("right")
ax3.yaxis.tick_right()

# Secondary electrons concurrent intensity vs position
ax4 = ax3.twiny()  # overlay on previous plot
sec_plot = ax4.plot(distances, sec, label='Secondary\nElectrons', linewidth=3, c='black')
ax4.set_ylim(np.amin(sec), np.amax(sec))
ax4.set_xlim(np.amin(distances), np.amax(distances))
ax4.xaxis.set_visible(False)
# ax4.set_xlabel('SEC Intensity)', c="C1")

# NMF Spectral Components
ax0 = fig.add_subplot(gs[-2:,0:3])
for i in range(num_NMF_components):
    ax0.plot(H_nmf[i], wavelengths, lw=2, label=f'{i+1}', c=cs[i], alpha=0.4)
ax0.plot(H_nmf[nmf_focus], wavelengths, lw=2, c=cs[nmf_focus])
ax0.set_ylim(wl_min,wl_max)
ax0.yaxis.set_visible(False)
ax0.set_xlabel('CL Intensity (a.u.)')
ax0.invert_xaxis()
ax0.legend(loc='upper left', bbox_to_anchor=(0,0.9))

# NMF Highlighted component (inset)
ax0b = fig.add_subplot(gs[2,1])
ax0b.plot(H_nmf[nmf_focus][sub_start:sub_end], wavelengths[sub_start:sub_end], lw=3, c=cs[nmf_focus])
ax0b.set_ylim(wl_min_subset, wl_max_subset)
ax0b.set_xlim(0,1)
ax0b.yaxis.set_visible(False)
ax0b.invert_xaxis()

# Cumulative CL intensity vs wavelength
ax1 = plt.subplot(gs[-2:,-2])
ax1.plot(w_sums, wavelengths)
ax1.fill_betweenx(y=wavelengths, x1=w_sums, x2=np.amin(w_sums)*0.9, alpha=0.3, color="C0", label="Cumulative Sum")
ax1.fill_betweenx(y=wavelengths[start:end], x1=w_sums[start:end], x2=np.amin(w_sums)*0.9, alpha=0.4, color="C3", label=f"{start_wl}–{end_wl} nm")
ax1.yaxis.set_visible(False)
#     ax1.set_xlim(np.amin(w_sums)*0.95, np.amax(w_sums)*1.05)
ax1.ticklabel_format(style='sci', scilimits=(1,3))
ax1.invert_xaxis()
ax1.set_ylim(wl_min,wl_max)
ax1.set_xlabel('Integrated\nCL Intensity (a.u.)')
# ax0.legend()

# Heatmap of spectra
ax2 = plt.subplot(gs[-2:,-1])
ax2.pcolormesh(XX, YY, spectra_corrected.T, shading='auto', cmap=cmap1)
ax2.set_xlabel('Distance along scan ($\mu$m)')
ax2.set_ylabel('Wavelength (nm)')
ax2.yaxis.set_label_position("right")
ax2.yaxis.tick_right()
ax2.set_ylim(wl_min,wl_max)

# Gather legend handles from relevant subplots
intensity_graphs = zip(ax1.get_legend_handles_labels(), ax4.get_legend_handles_labels())
handles, labels = [(a + b) for a, b in intensity_graphs]
fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(0.29,0.713), fontsize="small", frameon=True, framealpha=0.7)

fig.patch.set_alpha(0)
fig_name = 'linescan_analysis'
fig_ext = 'png'
fig_path = ''.join((fig_dir, '/', fig_name, '.', fig_ext))
# fig.savefig(fig_path, format=fig_ext, dpi=300)
# print(f'Figure saved to {fig_path}')

# Display plot
# plt.tight_layout()
plt.show()

## Orientation: Vertical linescan

In [ ]:
# Generate composite figure

wl_min = 350  # nm, x_min for displayed wavelength axis
wl_max = 950  # nm, x_max for displayed wavelength axis
# cmap1 = mpl.cm.get_cmap('gist_stern')
cmap1 = mpl.cm.get_cmap('RdBu_r')
nmf_focus = 1
start_wl = 750
end_wl = 900
threshold = 40

# Convert 'start_wl' and 'end_wl' to raw index values of wavelengths array
start = find_nearest(wavelengths, start_wl)
end = find_nearest(wavelengths, end_wl)

# Gather secondary electron survey data
ses = get_img_arr(f['Acquisition0'])
(ny, nx) = np.shape(ses)  # number of pixels (y,x) of SEM image
ses_x_step = get_scaleX(f['Acquisition0'])
ses_y_step = get_scaleY(f['Acquisition0'])

# Gather CL acquisition reference parameters
cl_acq = f['Acquisition2']
x_center = ses_x_step*nx/2
y_center = ses_y_step*ny/2
y_rng, x_rng = get_ranges(cl_acq, mult_scale=1)  # This is backwards from what I expect, but plots correctly
cl_x = [x_center + np.amin(x_rng), x_center + np.amax(x_rng)]
cl_y = [y_center + np.amin(y_rng), y_center + np.amax(y_rng)]

x_img = np.linspace(0, nx*ses_x_step, nx)  # Raw distance of image width
y_img = np.linspace(0, ny*ses_y_step, ny)  # Raw distance of image height
xx, yy = np.meshgrid(x_img, y_img)


# Generate x-axis scale and total wavelength sums
# wavelengths = get_scaleC(f['Acquisition2'])*1e9
# w_sums = [np.sum(i) for i in spectra_corrected]
# w_sums = np.array(w_sums)
w_sums = rescale(w_sums)

# Generate cummulative intensity for all wavelengths between start and end
I_tot = np.array([line.sum() for line in spectra_corrected])  # Sum the intensities over total wavelengths
I_sub = np.array([line[start:end].sum() for line in spectra_corrected])  # Sum the intensities in a given index range
I_tot = normalize(I_tot)
I_sub = normalize(I_sub)
# I_tot = I_tot/1e3
# I_sub = I_sub/1e3

# Pick out shape parameters from acquisition array, then reshape
# w_width = cl_full.shape[0]  # wavelengths, size 1024 for CCD pixels on spectrometer
# y_width = cl_full.shape[1]  # Number of acquisitions in horizontal direction (even though says y)
# x_width = cl_full.shape[2]
# pixels = y_width * x_width

# Generate colormesh params from file metadata
# step_x = get_scaleX(f['Acquisition2'])  # distance b/w pixels, in m
# distances = np.array([i*step_x for i in range(len(spectra_corrected[0]))])*1e6  # populate scaled x-axis
XX, YY = np.meshgrid(wavelengths, distances)

# Specify parameters for highlighted subset of NMF components (graph inset)
wl_min_subset = wl_min + (1/3 - 0.03)*(wl_max - wl_min)
wl_max_subset = wl_max
sub_start = find_nearest(wavelengths, wl_min_subset)
sub_end = find_nearest(wavelengths, wl_max_subset)

# temp_settings = {"font.size" : 9}
temp_settings = {}
# START PLOT -------------------------------

fig = plt.figure(figsize=(10,8))
# gs = GridSpec(ncols=4, nrows=4, figure=fig, height_ratios=[4,1,0.5,0.5], width_ratios=[1,3,1,1])
# fig = plt.figure(figsize=(6,4))
gs = GridSpec(ncols=4, nrows=4, figure=fig, height_ratios=[4,1,0.7,0.3], width_ratios=[1,3,1,1])
# fig.suptitle('CL Spectra')

with plt.rc_context(temp_settings):
    
    # Heatmap of spectra
    ax0 = plt.subplot(gs[0,0:-2])
    heatmap = ax0.pcolormesh(XX, YY, spectra_corrected, shading='auto', cmap=cmap1)
    ax0.set_ylabel('Distance Along Scan ($\mu$m)')
    ax0.set_xlabel('Wavelength (nm)')
    ax0.set_xlim(wl_min,wl_max)
    ax0.xaxis.set_label_position('top')
    ax0.tick_params(labeltop=True, labelbottom=False)

    # Cummulative CL intensity vs position
    ax1 = plt.subplot(gs[0,-2])
    # ax1.yaxis.set_visible(False)
    ax1.tick_params(labelleft=False)
    # ax1.plot(I_sub, distances, label=f'From {start_wl} to {end_wl}', linewidth=0.1, c="C3")  # Integrated wavelengths
    ax1.fill_betweenx(y=distances, x1=I_sub, x2=I_tot, color="C3", alpha=0.4)  # Int wavelengths
    ax1.fill_betweenx(y=distances, x1=I_tot, x2=np.amin(I_tot), alpha=0.4)  # All wavelengths
    ax1.set_ylim(np.amin(distances), np.amax(distances))
    ax1.set_xlim(np.amin(I_sub), np.amax(I_sub))
    # ax1.set_xlabel('Normalized\nintensity')
    ax1.xaxis.set_visible(False)
    ax1.xaxis.set_label_position('top')

    # Secondary electrons concurrent intensity vs position
    ax1b = ax1.twiny()  # overlay on previous plot
#     sec_plot = ax1b.plot(sec, distances, label='S.E.', lw=1, c='black')
    sec_plot = ax1b.plot(sec, distances, label='S.E.', c='black')
    ax1b.set_xlim(np.amin(sec), np.amax(sec))
    # ax1b.xaxis.set_visible(False)
    ax1b.set_xlabel('Normalized\nIntensity')
    ax1b.tick_params(labeltop=True, labelbottom=False)
    ax1b.xaxis.set_label_position('top')
#     ax1b.legend(fontsize='x-small', bbox_to_anchor=(0.4,0.4), handlelength=0.5)
    ax1b.legend(fontsize='small', bbox_to_anchor=(0.4,0.4), handlelength=0.8)

    # NMF Spatial Components
    ax2 = fig.add_subplot(gs[0, -1])
    # nmf_c1.imshow(W_nmf[:,0].reshape(y_width, x_width), cmap=reds)
    for i in range(num_NMF_components):
#         ax2.plot(W_nmf[:,i].reshape(y_width, x_width), distances, label=f'{i+1}', c=cs[i], lw=1, alpha=0.4)
        ax2.plot(W_nmf[:,i].reshape(y_width, x_width), distances, label=f'{i+1}', c=cs[i], alpha=0.4)
    ax2.plot(W_nmf[:,nmf_focus].reshape(y_width, x_width), distances, c=cs[nmf_focus], lw=1)
    ax2.set_ylim(np.amin(distances), np.amax(distances))
    # ax2.set_ylabel('Distance along scan ($\mu$ m)')
    ax2.set_xlabel('NMF Comp.\nStrength (a.u.)')
    ax2.tick_params(labeltop=True, labelbottom=False, labelleft=False)
    ax2.xaxis.set_label_position('top')
#     ax2.legend(fontsize='x-small')
    ax2.legend(fontsize='small')

    # Cumulative CL intensity vs wavelength
    ax3 = plt.subplot(gs[1,0:-2])
    ax3.plot(wavelengths, w_sums)
    ax3.fill_between(wavelengths, w_sums, np.amin(w_sums)*0.9, alpha=0.3, color="C0", label="Cumulative Sum")
    ax3.fill_between(x=wavelengths[start:end], y1=w_sums[start:end], y2=np.amin(w_sums)*0.9, alpha=0.4, color="C3", label=f"{start_wl}--{end_wl} nm")
    # ax3.xaxis.set_visible(False)
    ax3.set_ylim(np.amin(w_sums)*0.90, np.amax(w_sums)*1.05)
    ax3.set_xlim(wl_min,wl_max)
    ax3.set_ylabel('Integrated\nCL Intensity\n(a.u.)')
    ax3.tick_params(labelbottom=False, labeltop=False)
#     ax3.legend(fontsize='x-small')
    ax3.legend(fontsize='small')

    # NMF Spectral Components
    ax4 = fig.add_subplot(gs[-2:, 0:-2])
    for i in range(num_NMF_components):
#         ax4.plot(wavelengths, H_nmf[i], lw=1, label=f'{i+1}', c=cs[i], alpha=0.4)
        ax4.plot(wavelengths, H_nmf[i], label=f'{i+1}', c=cs[i], alpha=0.4)
#     ax4.plot(wavelengths, H_nmf[nmf_focus], lw=1, c=cs[nmf_focus])
    ax4.plot(wavelengths, H_nmf[nmf_focus], c=cs[nmf_focus])
    ax4.set_xlim(wl_min,wl_max)
    # ax4.xaxis.set_visible(False)
    ax4.tick_params(labelbottom=False, labeltop=False)
    ax4.set_ylabel('CL\nIntensity\n(a.u.)')
    # ax4.set_ylim(-50,1700)
    # ax4.legend()

    ax4b = fig.add_subplot(gs[-2:-1,1:-2])
#     ax4b.plot(wavelengths[sub_start:sub_end], H_nmf[nmf_focus][sub_start:sub_end], lw=1, c=cs[nmf_focus])
    ax4b.plot(wavelengths[sub_start:sub_end], H_nmf[nmf_focus][sub_start:sub_end], c=cs[nmf_focus])
    ax4b.set_xlim(wl_min_subset, wl_max_subset)
    ax4b.tick_params(labelbottom=False)
    ax4b.set_ylim(0,1.2)

    # Secondary Electron Survey with Line Scan
    ax5 = fig.add_subplot(gs[-3:,-2:])
    ax5.pcolormesh(xx, yy, ses, cmap='gray', shading='auto', alpha=0, vmin=np.amin(ses)*1.02, vmax=np.amax(ses)*0.98)  # SEM image
    # ax5.imshow(ses, cmap='gray', alpha=0.4)  # SEM image
#     ax5.plot(cl_x, cl_y, c="white")  # Line overlay of acquisition region
    ax5.plot(cl_x, cl_y, linewidth=4, c="white")  # Line overlay of acquisition region
    ax5.invert_yaxis()
    ax5.axis('equal')  # Force aspect ratio for SEM image
    ax5.axis('off')


fig.patch.set_alpha(0)
# fig_name = 'linescan_analysis_alt'
fig_name = 'poster_linescan_analysis_alt'
fig_ext = 'png'
fig_path = ''.join((fig_dir, '/', fig_name, '.', fig_ext))
fig.savefig(fig_path, format=fig_ext, dpi=300)
print(f'Figure saved to {fig_path}')

# Display plot
# plt.tight_layout()
plt.show()

In [ ]:
# User-supplied tweaks
# start_wl = 750  # Begin integration wavelength, in nm
# end_wl = 900    # Cutoff integration wavelength, in nm
# threshold = 30  # for cosmic ray removal (smaller is stricter)
# nmf_focus = 4  # NMF component to highlight (0 indexed)

def generate_plots(f, start_wl, end_wl, threshold, nmf_focus):

    # IMPORT AND FORMAT DATA ------------------------------
    wl_min = 350  # nm, x_min for displayed wavelength axis
    wl_max = 950  # nm, x_max for displayed wavelength axis
    num_NMF_components = 6  # for NMF decomposition
    cmap1 = mpl.cm.get_cmap('gist_stern')

 
    # Get data from file
    print(f) # Display filename above graph for reference
    cl = get_img_arr(f['Acquisition2'])  # size (1024,n) array
    cl = np.array([remove_spikes(s, threshold=threshold) for s in cl])
    cl_full = np.atleast_3d(cl)
    sec = get_img_arr(f['Acquisition1'])  # Secondary electrons concurrent
    sec = normalize(sec)
    # sec = sec/1e3
    
    # Generate x-axis scale and total wavelength sums
    wavelengths = get_scaleC(f['Acquisition2'])*1e9
    w_sums = [np.sum(i) for i in cl]
    w_sums = np.array(w_sums)/1e6
    
    # Pick out shape parameters from acquisition array, then reshape
    w_width = cl_full.shape[0]  # wavelengths, size 1024 for CCD pixels on spectrometer
    y_width = cl_full.shape[1]  # Number of acquisitions in horizontal direction (even though says y)
    x_width = cl_full.shape[2]
    pixels = y_width * x_width
    spectra = cl_full.transpose(1,2,0).reshape(pixels, w_width)  # Reshape so that row is one spectrum
    orig_spectra = cl_full.transpose(1,2,0).reshape(pixels, w_width)
    med_sub_orig_spectra = np.absolute(spectra - np.median(spectra))

    # Convert 'start_wl' and 'end_wl' to raw index values of wavelengths array
    start = find_nearest(wavelengths, start_wl)
    end = find_nearest(wavelengths, end_wl)

    # Generate colormesh params from file metadata
    step_x = get_scaleX(f['Acquisition2'])  # distance b/w pixels, in m
    distances = np.array([i*step_x for i in range(len(cl[0]))])*1e6  # populate scaled x-axis
    XX, YY = np.meshgrid(wavelengths, distances)

    # Generate cummulative intensity for all wavelengths between start and end
    I_tot = np.array([line.sum() for line in spectra])  # Sum the intensities over total wavelengths
    I_sub = np.array([line[start:end].sum() for line in spectra])  # Sum the intensities in a given index range
    I_tot = normalize(I_tot)
    I_sub = normalize(I_sub)
    # I_tot = I_tot/1e3
    # I_sub = I_sub/1e3

    # NMF ANALYSIS ------------------------------ 
    
    
    denoise_kwargs = dict(multichannel=False,  convert2ycbcr=False, wavelet='sym8',
                          rescale_sigma=False, wavelet_levels=6,    mode='soft')

    bhigh = .000005
    blow = .75
    bdeg = 5

    bf_sos_h = butter(bdeg, bhigh, btype='high', analog=False, output='sos')

    f_pre_spectra = [median_filter(spectrum, size=(20)) for spectrum in spectra]
    fhspectra = np.array([sosfiltfilt(bf_sos_h, spectrum) for spectrum in f_pre_spectra])
    fspectra = [denoise_wavelet(s, **denoise_kwargs)*65535 for s in fhspectra]

    fspectra = [spectrum.clip(min=0) for spectrum in fspectra]

    nmf_model = NMF(n_components=num_NMF_components , init='random', max_iter=5000, random_state=5)

    W_nmf = nmf_model.fit_transform(fspectra)
    H_nmf = nmf_model.components_
    for i in range(num_NMF_components):
#         W_nmf[:,i] = remove_spikes(W_nmf[:,i], threshold=60)
        # Plots are still too rough, smooth them out
        W_nmf[:,i] = gaussian_filter1d(W_nmf[:,i], sigma=2)
        
    print('NMF Error', nmf_model.reconstruction_err_)


#     fig2 = plt.figure(constrained_layout=False, figsize=(10,8))
#     grid_two = GridSpec(ncols=2, nrows=6, figure=fig2, width_ratios=[1,3])
    # fig2.suptitle(mtitle1 + "\nNMF Components (Cleaned Hyperspectrum)", fontsize=12, y=.95)


    # START PLOT -------------------------------
    
#     fig = plt.figure(figsize=(10,8))
    gs = GridSpec(ncols=3, nrows=3, figure=fig, height_ratios=[2,1,4], width_ratios=[4,1,1])
#     fig.suptitle('CL Spectra')

    # NMF Spectral Components
    ax0 = fig.add_subplot(gs[0, 0])
    for i in range(num_NMF_components):
        ax0.plot(wavelengths, H_nmf[i], lw=4, label='Component {}'.format(i+1), c=cs[i], alpha=0.4)
    ax0.plot(wavelengths, H_nmf[nmf_focus], lw=4, c=cs[nmf_focus])
    ax0.set_xlim(wl_min,wl_max)
    ax0.xaxis.set_visible(False)
    ax0.set_ylabel('CL Intensity\n(a.u.)')
#     ax0.set_ylim(-50,1700)
    ax0.legend()

    # Cumulative CL intensity vs wavelength
    ax1 = plt.subplot(gs[1,0])
    ax1.plot(wavelengths, w_sums)
    ax1.fill_between(wavelengths, w_sums, np.amin(w_sums)*0.9, alpha=0.3, color="C0", label="Cumulative Sum")
    ax1.fill_between(x=wavelengths[start:end], y1=w_sums[start:end], y2=np.amin(w_sums)*0.9, alpha=0.4, color="C3", label=f"{start_wl} to {end_wl} nm")
    ax1.xaxis.set_visible(False)
    ax1.set_ylim(np.amin(w_sums)*0.95, np.amax(w_sums)*1.05)
    ax1.set_xlim(wl_min,wl_max)
    ax1.set_ylabel('Integrated\nCL Intensity (a.u.)')
    # ax0.legend()

    # Heatmap of spectra
    ax2 = plt.subplot(gs[2,0])
    heatmap = ax2.pcolormesh(XX, YY, cl.T, shading='auto', cmap=cmap1)
    ax2.set_ylabel('Distance along scan ($\mu$m)')
    ax2.set_xlabel('Wavelength (nm)')
    ax2.set_xlim(wl_min,wl_max)

    # Cummulative CL intensity vs position
    ax3 = plt.subplot(gs[2,1])
    ax3.yaxis.set_visible(False)
#     ax2.plot(I_sub, distances, label=f'From {start_wl} to {end_wl}', linewidth=0.1, c="C3")  # Integrated wavelengths
    ax3.fill_betweenx(y=distances, x1=I_sub, x2=I_tot, color="C3", alpha=0.4)  # Int wavelengths
    ax3.fill_betweenx(y=distances, x1=I_tot, x2=np.amin(I_tot), alpha=0.4)  # All wavelengths
    ax3.set_ylim(np.amin(distances), np.amax(distances))
    ax3.set_xlim(np.amin(I_sub), np.amax(I_sub))
    ax3.set_xlabel('Normalized\nintensity')

    # Secondary electrons concurrent intensity vs position
    ax4 = ax3.twiny()  # overlay on previous plot
    sec_plot = ax4.plot(sec, distances, label='SEC intensity', linewidth=3, c='black')
    ax4.set_xlim(np.amin(sec), np.amax(sec))
    ax4.xaxis.set_visible(False)
    # ax4.set_xlabel('SEC Intensity)', c="C1")
    
    # NMF Satial Components
    ax5 = fig.add_subplot(gs[2, 2])
#     nmf_c1.imshow(W_nmf[:,0].reshape(y_width, x_width), cmap=reds)
    for i in range(num_NMF_components):
        ax5.plot(W_nmf[:,i].reshape(y_width, x_width), distances, c=cs[i], linewidth=3, alpha=0.4)
    ax5.plot(W_nmf[:,nmf_focus].reshape(y_width, x_width), distances, c=cs[nmf_focus], linewidth=3)
    ax5.set_ylim(np.amin(distances), np.amax(distances))
#     ax5.set_ylabel('Distance along scan ($\mu$ m)')
    ax5.yaxis.set_visible(False)
    ax5.set_xlabel('NMF Comp.\nStrength (a.u.)')
#     ax5.legend()
    

    # Gather legend handles from relevant subplots
    intensity_graphs = zip(ax1.get_legend_handles_labels(), ax4.get_legend_handles_labels())
    handles, labels = [(a + b) for a, b in intensity_graphs]
    fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.75,0.7))
    
#     nmf_graphs = zip(ax0.get_legend_handles_labels(), ax5.get_legend_handles_labels())
#     handles, labels = [(a + b) for a, b in nmf_graphs]
#     fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.97,0.97))

    # Display plot
#     plt.tight_layout()
#     plt.show()
    return fig

In [ ]:
# User-supplied tweaks
# start_wl = 750  # Begin integration wavelength, in nm
# end_wl = 900    # Cutoff integration wavelength, in nm
# threshold = 30  # for cosmic ray removal (smaller is stricter)
# nmf_focus = 4  # NMF component to highlight (0 indexed)

# Same function as above, but experimenting with tweaked layout
#  - Swapped wavelength / distance axes
#  - Swapped corresponding sums from each axis
#  - Moved whitespace to top left instead of top right
#  - Return fig object instead of forcing plot, enables adjustments after calling

def generate_plots(f, start_wl, end_wl, threshold, nmf_focus):

    # IMPORT AND FORMAT DATA ------------------------------
    wl_min = 350  # nm, x_min for displayed wavelength axis
    wl_max = 950  # nm, x_max for displayed wavelength axis
    num_NMF_components = 6  # for NMF decomposition
#     cmap1 = mpl.cm.get_cmap('gist_stern')
    cmap1 = mpl.cm.get_cmap('RdBu_r')

 
    # Get data from file
    print(f) # Display filename above graph for reference
    cl = get_img_arr(f['Acquisition2'])  # size (1024,n) array
    cl = np.array([remove_spikes(s, threshold=threshold) for s in cl])
    cl_full = np.atleast_3d(cl)
    sec = get_img_arr(f['Acquisition1'])  # Secondary electrons concurrent
    sec = normalize(sec)
    # sec = sec/1e3
    
    # Gather secondary electron survey data
    ses = get_img_arr(f['Acquisition0'])
    (ny, nx) = np.shape(ses)  # number of pixels (y,x) of SEM image
    ses_x_step = get_scaleX(f['Acquisition0'])
    ses_y_step = get_scaleY(f['Acquisition0'])

    # Gather CL acquisition reference parameters
    cl_acq = f['Acquisition2']
    x_center = ses_x_step*nx/2
    y_center = ses_y_step*ny/2
    y_rng, x_rng = get_ranges(cl_acq, mult_scale=1)  # This is backwards from what I expect, but plots correctly
    cl_x = [x_center + np.amin(x_rng), x_center + np.amax(x_rng)]
    cl_y = [y_center + np.amin(y_rng), y_center + np.amax(y_rng)]
    
    x_img = np.linspace(0, nx*ses_x_step, nx)  # Raw distance of image width
    y_img = np.linspace(0, ny*ses_y_step, ny)  # Raw distance of image height
    xx, yy = np.meshgrid(x_img, y_img)


    # Generate x-axis scale and total wavelength sums
    wavelengths = get_scaleC(f['Acquisition2'])*1e9
    w_sums = [np.sum(i) for i in cl]
    w_sums = np.array(w_sums)
    w_sums = rescale(w_sums)
    
    # Pick out shape parameters from acquisition array, then reshape
    w_width = cl_full.shape[0]  # wavelengths, size 1024 for CCD pixels on spectrometer
    y_width = cl_full.shape[1]  # Number of acquisitions in horizontal direction (even though says y)
    x_width = cl_full.shape[2]
    pixels = y_width * x_width
    spectra = cl_full.transpose(1,2,0).reshape(pixels, w_width)  # Reshape so that row is one spectrum
    orig_spectra = cl_full.transpose(1,2,0).reshape(pixels, w_width)
    med_sub_orig_spectra = np.absolute(spectra - np.median(spectra))

    # Convert 'start_wl' and 'end_wl' to raw index values of wavelengths array
    start = find_nearest(wavelengths, start_wl)
    end = find_nearest(wavelengths, end_wl)
    
    # Specify parameters for highlighted subset of NMF components (graph inset)
    wl_min_subset = wl_min + (1/3 - 0.02)*(wl_max - wl_min)
    wl_max_subset = wl_max
    sub_start = find_nearest(wavelengths, wl_min_subset)
    sub_end = find_nearest(wavelengths, wl_max_subset)

    # Generate colormesh params from file metadata
    step_x = get_scaleX(f['Acquisition2'])  # distance b/w pixels, in m
    distances = np.array([i*step_x for i in range(len(cl[0]))])*1e6  # populate scaled x-axis
    XX, YY = np.meshgrid(distances, wavelengths)

    # Generate cummulative intensity for all wavelengths between start and end
    I_tot = np.array([line.sum() for line in spectra])  # Sum the intensities over total wavelengths
    I_sub = np.array([line[start:end].sum() for line in spectra])  # Sum the intensities in a given index range
    I_tot = normalize(I_tot)
    I_sub = normalize(I_sub)
    # I_tot = I_tot/1e3
    # I_sub = I_sub/1e3

    # NMF ANALYSIS ------------------------------ 
    
    
    denoise_kwargs = dict(multichannel=False,  convert2ycbcr=False, wavelet='sym8',
                          rescale_sigma=False, wavelet_levels=6,    mode='soft')

    bhigh = .000005
    blow = .75
    bdeg = 5

    bf_sos_h = butter(bdeg, bhigh, btype='high', analog=False, output='sos')

    f_pre_spectra = [median_filter(spectrum, size=(20)) for spectrum in spectra]
    fhspectra = np.array([sosfiltfilt(bf_sos_h, spectrum) for spectrum in f_pre_spectra])
    fspectra = [denoise_wavelet(s, **denoise_kwargs)*65535 for s in fhspectra]

    fspectra = [spectrum.clip(min=0) for spectrum in fspectra]

    nmf_model = NMF(n_components=num_NMF_components , init='random', max_iter=5000, random_state=5)

    W_nmf = nmf_model.fit_transform(fspectra)
    H_nmf = nmf_model.components_
    for i in range(num_NMF_components):
#         W_nmf[:,i] = remove_spikes(W_nmf[:,i], threshold=60)
        # Plots are still too rough, smooth them out
        W_nmf[:,i] = gaussian_filter1d(W_nmf[:,i], sigma=2)
        
    print('NMF Error', nmf_model.reconstruction_err_)


#     fig2 = plt.figure(constrained_layout=False, figsize=(10,8))
#     grid_two = GridSpec(ncols=2, nrows=6, figure=fig2, width_ratios=[1,3])
    # fig2.suptitle(mtitle1 + "\nNMF Components (Cleaned Hyperspectrum)", fontsize=12, y=.95)


    # START PLOT -------------------------------
    
    fig = plt.figure(figsize=(10,8))
    gs = GridSpec(ncols=5, nrows=4, figure=fig, height_ratios=[3,3,9,3], width_ratios=[1,1.25,1.75,2,12])
#     fig.suptitle('CL Spectra')

    # Secondary Electron Survey with Line Scan
    ax6 = fig.add_subplot(gs[0:2,0:-1])
    ax6.pcolormesh(xx, yy, ses, cmap='gray', shading='auto', alpha=0, vmin=np.amin(ses)*1.02, vmax=np.amax(ses)*0.98)  # SEM image
#     ax6.imshow(ses, cmap='gray', alpha=0.4)  # SEM image
    ax6.plot(cl_x, cl_y, linewidth=3, c="white")  # Line overlay of acquisition region
    ax6.invert_yaxis()
    ax6.axis('equal')  # Force aspect ratio for SEM image
    ax6.axis('off')

    # NMF Spatial Components
    ax5 = fig.add_subplot(gs[0,-1])
    for i in range(num_NMF_components):
        ax5.plot(distances, W_nmf[:,i].reshape(y_width, x_width), c=cs[i], linewidth=3, alpha=0.4)
    ax5.plot(distances, W_nmf[:,nmf_focus].reshape(y_width, x_width), c=cs[nmf_focus], linewidth=3)
    ax5.set_xlim(np.amin(distances), np.amax(distances))
    ax5.set_xlabel('Distance along scan ($\mu$ m)')
    ax5.xaxis.set_visible(False)
    ax5.set_ylabel('NMF Comp.\nStrength (a.u.)')
    ax5.yaxis.set_label_position("right")
    ax5.yaxis.tick_right()
#     ax5.legend()

    # Cummulative CL intensity vs position
    ax3 = plt.subplot(gs[1,-1])
    ax3.fill_between(x=distances, y1=I_sub, y2=I_tot, color="C3", alpha=0.4)  # Int wavelengths
    ax3.fill_between(x=distances, y1=I_tot, y2=np.amin(I_tot), alpha=0.4)  # All wavelengths
    ax3.set_xlim(np.amin(distances), np.amax(distances))
    ax3.xaxis.set_visible(False)
    ax3.set_ylim(np.amax(I_sub), np.amin(I_sub))
    ax3.set_ylabel('Normalized\nIntensity')
    ax3.yaxis.set_label_position("right")
    ax3.yaxis.tick_right()

    # Secondary electrons concurrent intensity vs position
    ax4 = ax3.twiny()  # overlay on previous plot
    sec_plot = ax4.plot(distances, sec, label='Secondary\nElectrons', linewidth=3, c='black')
    ax4.set_ylim(np.amin(sec), np.amax(sec))
    ax4.set_xlim(np.amin(distances), np.amax(distances))
    ax4.xaxis.set_visible(False)
    # ax4.set_xlabel('SEC Intensity)', c="C1")

    # NMF Spectral Components
    ax0 = fig.add_subplot(gs[-2:,0:3])
    for i in range(num_NMF_components):
        ax0.plot(H_nmf[i], wavelengths, lw=4, label=f'{i+1}', c=cs[i], alpha=0.4)
    ax0.plot(H_nmf[nmf_focus], wavelengths, lw=4, c=cs[nmf_focus])
    ax0.set_ylim(wl_min,wl_max)
    ax0.yaxis.set_visible(False)
    ax0.set_xlabel('CL Intensity (a.u.)')
    ax0.invert_xaxis()
    ax0.legend(loc='upper left', bbox_to_anchor=(0,0.9))
    
    # NMF Highlighted component (inset)
    ax0b = fig.add_subplot(gs[2,1])
    ax0b.plot(H_nmf[nmf_focus][sub_start:sub_end], wavelengths[sub_start:sub_end], lw=4, c=cs[nmf_focus])
    ax0b.set_ylim(wl_min_subset, wl_max_subset)
    ax0b.yaxis.set_visible(False)
    ax0b.invert_xaxis()

    # Cumulative CL intensity vs wavelength
    ax1 = plt.subplot(gs[-2:,-2])
    ax1.plot(w_sums, wavelengths)
    ax1.fill_betweenx(y=wavelengths, x1=w_sums, x2=np.amin(w_sums)*0.9, alpha=0.3, color="C0", label="Cumulative Sum")
    ax1.fill_betweenx(y=wavelengths[start:end], x1=w_sums[start:end], x2=np.amin(w_sums)*0.9, alpha=0.4, color="C3", label=f"{start_wl}–{end_wl} nm")
    ax1.yaxis.set_visible(False)
#     ax1.set_xlim(np.amin(w_sums)*0.95, np.amax(w_sums)*1.05)
    ax1.ticklabel_format(style='sci', scilimits=(1,3))
    ax1.invert_xaxis()
    ax1.set_ylim(wl_min,wl_max)
    ax1.set_xlabel('Integrated\nCL Intensity (a.u.)')
    # ax0.legend()

    # Heatmap of spectra
    ax2 = plt.subplot(gs[-2:,-1])
    ax2.pcolormesh(XX, YY, cl, shading='auto', cmap=cmap1)
    ax2.set_xlabel('Distance along scan ($\mu$m)')
    ax2.set_ylabel('Wavelength (nm)')
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    ax2.set_ylim(wl_min,wl_max)

    
    # Gather legend handles from relevant subplots
    intensity_graphs = zip(ax1.get_legend_handles_labels(), ax4.get_legend_handles_labels())
    handles, labels = [(a + b) for a, b in intensity_graphs]
    fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(0.29,0.713), fontsize="small", frameon=True, framealpha=0.7)
    
#     nmf_graphs = zip(ax0.get_legend_handles_labels(), ax5.get_legend_handles_labels())
#     handles, labels = [(a + b) for a, b in nmf_graphs]
#     fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.97,0.97))

    # Display plot
#     plt.tight_layout()
#     plt.show()
    return fig

In [ ]:
start_wl = 750
end_wl = 900
threshold = 40
nmf_focus = [4,4,4,4,0,0,4,4,4,4,4,4]

for i,f in enumerate(files[0:1]):
    h5 = h5py.File(f, 'r')
    fig = generate_plots(h5, start_wl, end_wl, threshold, nmf_focus[i])
    fig.patch.set_alpha(0)
    fig_name = 'linescan_analysis'
    fig_ext = 'png'
    fig_path = ''.join((fig_dir, '/', fig_name, '.', fig_ext))
#     fig.savefig(fig_path, format=fig_ext, dpi=300)
#     print(f'Figure saved to {fig_path}')
    plt.show()

### Secondary Electrons Concurrent vs Position:

Back-scattered electrons gathered from the electron-beam interaction with the sample at each pixel, obtained concurrently with the acquired CL spectrum.

In [ ]:
sec = get_img_arr(f['Acquisition1'])  # Array of size (n,)
sec = remove_spikes(sec, threshold=5)  # Adjust threshold as necessary
step_x = get_scaleX(f['Acquisition1'])  # distance in m
distances = [i*step_x for i in range(len(sec))]  # populate scaled x-axis
plt.plot(distances, sec)
plt.title('Secondary Electrons Concurrent Along Line Scan')
plt.ylabel('Intensity (counts)')
plt.xlabel('Distance (m)')
plt.show()

### CL Spectrum

Photons gathered from the electron-beam interaction with the sample, binned by wavelength.

### CL Intensity vs Position:

Photons gathered from the electron-beam interaction with the sample at each pixel, summed over a specified wavelength range by selecting the appropriate summing index, then plotted as an overall intensity per position.

In [ ]:
# User-supplied tweaks
start_wl = 750  # Begin integration wavelength, in nm
end_wl = 900    # Cutoff integration wavelength, in nm
threshold = 30  # for cosmic ray removal (smaller is stricter)

# Load data and clean it
cl = get_img_arr(f['Acquisition2'])  # The complete spectrum at each pixel; size (1024,n)
cl = np.array([remove_spikes(s, threshold=threshold) for s in cl])
wavelengths = get_scaleC(f['Acquisition2'])  # size (1024,) array, values in m
step_x = get_scaleX(f['Acquisition2'])  # distance b/w pixels, in m
distances = [i*step_x for i in range(len(cl[0]))]  # populate scaled x-axis

# Convert 'start' and 'end' to raw index values of wavelengths array
start = find_nearest(wavelengths, start_wl*1e-9)
end = find_nearest(wavelengths, end_wl*1e-9)
# Generate cummulative intensity for all wavelengths between start and end
I = np.array([line[start:end].sum() for line in cl.T])  # Sum the intensities in a given index range

plt.plot(distances,I)
plt.title(f'Integrated CL intensity {start_wl}-{end_wl} nm')
plt.ylabel('Intensity (counts)')
plt.xlabel('Distance (m)')

# Display plot
plt.show()

# Combined plots

### Plot both normalized lines together:

In [ ]:
# Call after the above two plots have been generated
plt.plot(distances, normalize(sec), label="Secondary Electrons Concurrent")
plt.plot(distances, normalize(I), label="Integrated CL Intensity")
plt.title('Normalized overlay of SEC and CL')
plt.ylabel('Intensity (a.u.)')
plt.xlabel('Distance (m)')
plt.legend()
plt.show()

### Plot an overlay of each graph

In [ ]:
fig, ax = plt.subplots()
fig.suptitle('Overlay of SEC and CL Intensity')
ax.plot(distances, sec, c='C0', linewidth=3)
ax.set_ylabel("Secondary Electrons Concurrent", c="C0")
ax.set_xlabel("Distance (m)")

ax2 = ax.twinx()
ax2.plot(distances, I, c='C1', alpha=0.8)
ax2.set_ylabel(f"Integrated CL ({start_wl}-{end_wl} nm)", c="C1")
plt.show()

Secondary electron survey:

In [ ]:
# Gather secondary electron survey data
ses = get_img_arr(f['Acquisition0'])
(ny, nx) = np.shape(ses)  # number of pixels (y,x) of SEM image
ses_x_step = get_scaleX(f['Acquisition0'])
ses_y_step = get_scaleY(f['Acquisition0'])

# Gather CL acquisition reference parameters
cl = f['Acquisition2']
x_center = ses_x_step*nx/2
y_center = ses_y_step*ny/2
y_rng, x_rng = get_ranges(cl, mult_scale=1)  # This is backwards from what I expect, but plots correctly
cl_x = [x_center + np.amin(x_rng), x_center + np.amax(x_rng)]
cl_y = [y_center + np.amin(y_rng), y_center + np.amax(y_rng)]

x_img = np.linspace(0, nx*ses_x_step, nx)  # Raw distance of image width
y_img = np.linspace(0, ny*ses_y_step, ny)  # Raw distance of image height
xx, yy = np.meshgrid(x_img, y_img)

plt.pcolormesh(xx, yy, ses, shading='auto', alpha=0.01)  # SEM image
plt.plot(cl_x, cl_y, linewidth=3, c="white")  # Line overlay of acquisition region
plt.gca().invert_yaxis()
plt.axis('off')
plt.show()

In [ ]:
xoffset = f['Acquisition1']['ImageData']['XOffset'][()]
xlen = len(get_img_arr(f['Acquisition1']))
step_x = get_scaleX(f['Acquisition1'])
yoffset = f['Acquisition1']['ImageData']['YOffset'][()]

In [ ]:
Title = f['Acquisition2']['PhysicalData']['Title'][()]
Title

# NMF

In [ ]:
def plot_NMF(h5):
#     hs_data = h5['Acquisition2']['ImageData']['Image'][()].squeeze()*1e-5             # Hyperspectral CL data
#     hs_data = np.atleast_3d(hs_data)
#     wavelengths = h5['Acquisition2']['ImageData']['DimensionScaleC'][()]*1e9     # The wavelength values in nanometers.
#     dim_x = h5['Acquisition2']['ImageData']['DimensionScaleX'][()]     

    pixels = hs_data.shape[1]*hs_data.shape[2]                                   # Number of pixels in the CL image
    spectra = hs_data.transpose(1,2,0).reshape(pixels, hs_data.shape[0])         # Reshape such that a row is one spectrum
    orig_spectra = hs_data.transpose(1,2,0).reshape(pixels, hs_data.shape[0])    # Copy of the above array
    med_sub_orig_spectra = np.absolute(spectra - np.median(spectra))             # Copy of spectra with median subtracted
#     print(hs_data.shape, spectra.shape, sem_image.shape, dim_x)
    step_x = get_scaleX(h5['Acquisition2'])  # distance b/w pixels, in m
    distances = np.array([i*step_x for i in range(len(hs_data[0]))])*1e6  # populate scaled x-axis

    x_width = hs_data.shape[2]
    y_width = hs_data.shape[1]

    denoise_kwargs = dict(multichannel=False,  convert2ycbcr=False, wavelet='sym8',
                          rescale_sigma=False, wavelet_levels=6,    mode='soft')

    bhigh = .000005
    blow = .75
    bdeg = 5

    bf_sos_h = butter(bdeg, bhigh, btype='high', analog=False, output='sos')

    f_pre_spectra = [median_filter(spectrum, size=(20)) for spectrum in spectra]
    fhspectra = np.array([sosfiltfilt(bf_sos_h, spectrum) for spectrum in f_pre_spectra])
    fspectra = [denoise_wavelet(s, **denoise_kwargs)*65535 for s in fhspectra]

    fspectra = [spectrum.clip(min=0) for spectrum in fspectra]

    num_components = 6  # NMF decomposition
    nmf_model = NMF(n_components=num_components , init='random', max_iter=5000, random_state=5)

    W_nmf = nmf_model.fit_transform(fspectra)
    H_nmf = nmf_model.components_
    for i in range(num_components):
#         W_nmf[:,i] = remove_spikes(W_nmf[:,i], threshold=60)
        # Plots are still too rough, smooth them out
        W_nmf[:,i] = gaussian_filter1d(W_nmf[:,i], sigma=2)
        
    print('NMF Error', nmf_model.reconstruction_err_)
    print('H_nmf:', H_nmf.shape)
    print('W_nmf:', W_nmf.shape)

    cmap1 = mpl.cm.get_cmap('viridis_r')

    fig2 = plt.figure(constrained_layout=False, figsize=(10,8))
    grid_two = GridSpec(ncols=2, nrows=6, figure=fig2, width_ratios=[1,3])
    # fig2.suptitle(mtitle1 + "\nNMF Components (Cleaned Hyperspectrum)", fontsize=12, y=.95)

    H_nmf_plt = fig2.add_subplot(grid_two[0:6, 0])
#     nmf_c1.imshow(W_nmf[:,0].reshape(y_width, x_width), cmap=reds)
    for i in range(num_components):
        H_nmf_plt.plot(W_nmf[:,i].reshape(y_width, x_width), distances, c=cs[i], label=f"{i}", linewidth=3)
    H_nmf_plt.set_ylim(np.amin(distances), np.amax(distances))
    H_nmf_plt.set_ylabel('Distance along scan ($\mu$ m)')
    H_nmf_plt.set_xlabel('NMF Component Strength (a.u.)')
    plt.legend()
    
    for i in range(num_components):
        nmf_sp = fig2.add_subplot(grid_two[i, 1])
        nmf_sp.plot(wavelengths, H_nmf[i], lw=4, label='Component {}'.format(i+1), c=cs[i])
    #     nmf_sp.set_ylabel('Weight (a.u.)', size=15)
        nmf_sp.set_xlim(300,900)
#         nmf_sp.set_ylim(0,1)
    #     nmf_sp.xaxis.set_visible(False)
        nmf_sp.legend()
    #nmf_sp.set_xlim(400,900)
    #nmf_sp.set_yticks([])
    nmf_sp.set_xlabel('Wavelength (nm)', size=15)
#     fig2.text(-0.02, 0.5, 'Weight (a.u.)', size=15, va='center', rotation='vertical')
    nmf_sp.set_xlim(300,900)
    # nmf_sp.set_ylim(-1,5)
    nmf_sp.legend()

    # plt.savefig(mtitle1 + "_NMF1_DIVDSPEC.png")

    plt.tight_layout()
    plt.show()

In [ ]:
for f in files[0:1]:
    h5 = h5py.File(f, 'r')
    print()
    plot_NMF(h5)